# Wrapping a subset of a very large library

Sometimes, for a very large library, only a subset of available *C++* components is useful for end-users.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider only a subset of the *C++* components during the `Generate` step.
The **Clang** library is a complete *C*/*C++* compiler.
**Clang** is a great tool, but its stable *Python* interface (i.e., **libclang**) is lacking some useful features that are needed by **AutoWIG**.
In particular, class template specializations are not available in the abstract syntax tree.
Fortunately, most of the classes that are needed during the traversal of the *C++* abstract syntax tree are not template specializations.
We therefore bootstrapped the **Clang** *Python* bindings using the `libclang` `parser` of **AutoWIG**.
This new **Clang** *Python* interface is called **ClangLite** and is able to parse class template specializations.
As for **libclang**, this interface is proposed only for a subset of the **Clang** library sufficient enough for proposing the new `clanglite` `parser`.


We here aim at presenting how subsets of very large libraries can be wrappred.
First, we need:

* to detect if the operating system (OS) is a Windows OS, a Mac OS or a linux OS.

In [1]:
import platform
is_windows = any(platform.win32_ver())
is_macosx = platform.sys.platform == 'darwin'

On Windows OSes, the visual studio version used to compile future wrappers must be given.

In [2]:
if is_windows:
    kwargs = dict(msvc_version = '14.0')
else:
    kwargs = dict()

* to detect the version of *Python* installed and to save it in the `PYTHON_VERSION` environment variable.

In [3]:
import sys
PYTHON_VERSION = str(sys.version_info.major) + '.' + str(sys.version_info.minor)

* to import **AutoWIG**.

In [4]:
import autowig

* to import **subprocess**.

In [5]:
import subprocess

* to detect the **Git** repository root

In [6]:
import os
GIT_ROOT = subprocess.check_output('git rev-parse --show-toplevel', shell=True).decode()
GIT_ROOT = GIT_ROOT.replace('/', os.sep).strip()
GIT_ROOT = os.path.join(GIT_ROOT, 'share', 'git', 'ClangLite')
from devops_tools import describe
os.environ['GIT_DESCRIBE_VERSION'] = describe.git_describe_version(GIT_ROOT)
os.environ['GIT_DESCRIBE_NUMBER'] = describe.git_describe_number(GIT_ROOT)
os.environ['DATETIME_DESCRIBE_VERSION'] = describe.datetime_describe_version(GIT_ROOT)
os.environ['DATETIME_DESCRIBE_NUMBER'] = describe.datetime_describe_number(GIT_ROOT)

Once these preliminaries are done, we can proceed to the actual generation of wrappers for the **Clang** & **ClangLite** libraries.
For this, we create an empty Abstract Semantic Graph (ASG).

In [7]:
asg = autowig.AbstractSemanticGraph()

We then parse the `tool.h` header of the **ClangLite** library with relevant compilation flags.

In [8]:
%%time

try:
    from path import path as Path
except:
    from path import Path
prefix = Path(sys.prefix).abspath()
if is_windows:
    headers = [prefix/'Library'/'include'/'clanglite'/'tool.h']
else:
    headers = [prefix/'include'/'clanglite'/'tool.h']
    
import six
if six.PY2 and is_windows:
    from autowig.libclang_parser import libclang_parser
    autowig.parser['libclang'] = libclang_parser
    autowig.parser.plugin = 'libclang'
    kwargs['silent'] = True
else:
    autowig.parser.plugin = 'clanglite'
    
flags = ['-x', 'c++', '-std=c++11',
         '-D__STDC_LIMIT_MACROS',
         '-D__STDC_CONSTANT_MACROS',
         '-DPYBIND11_GENERATOR']

if is_windows:
    flags.append('-I' + str((prefix/'Library'/'include').abspath()))
    flags.append('-I' + str((prefix/'include').abspath()))
else:
    flags.append('-I' + str((prefix/'include').abspath()))
    flags.append('-I' + str((prefix/'include'/'python3.7m').abspath()))
    
asg = autowig.parser(asg,
                     headers,
                     flags = flags,
                     bootstrap = False,
                     **kwargs)

/home/pfernique/Desktop/miniconda/envs/fp17/lib/python3.7/site-packages/clanglite/autowig_parser.py:896: Warning: 
  warnings.warn('', Warning)
/home/pfernique/Desktop/miniconda/envs/fp17/lib/python3.7/site-packages/clanglite/autowig_parser.py:898: Warning: kind.FUNCTION
  warnings.warn(str(kind), Warning)


CPU times: user 18min 58s, sys: 3.18 s, total: 19min 2s
Wall time: 19min 3s


Since most of **AutoWIG** guidelines are respected in the **Clang** & **ClangLite** libraries, the `default` `controller` implementation could be suitable.
Nevertheless, we need to force some *C++* components to be wrapped or not (in particular **Clang** components).
We therefore implements a new `controller`.

In [9]:
def clanglite_controller(asg):
    for node in asg.classes():
        node.pybind11_export = False
    for node in asg.functions(free = True):
        node.pybind11_export = False
    for node in asg.variables(free = True):
        node.pybind11_export = False
    for node in asg.enumerations():
        node.pybind11_export = False
    for node in asg.enumerators():
        if node.parent.pybind11_export:
            node.pybind11_export = False
    for node in asg.typedefs():
        node.pybind11_export = False
            
    from autowig.default_controller import refactoring
    asg = refactoring(asg)

    if autowig.parser.plugin == 'libclang':
        for fct in asg.functions(free = False):
            asg._nodes[fct._node]['_is_virtual'] = False
            asg._nodes[fct._node]['_is_pure'] = False
        asg['class ::clang::QualType'].is_abstract = False
        asg['class ::clang::QualType'].is_copyable = True
        asg['class ::llvm::StringRef'].is_abstract = False
        asg['class ::llvm::StringRef'].is_copyable = True
        asg['class ::clang::FileID'].is_abstract = False
        asg['class ::clang::FileID'].is_copyable = True
        asg['class ::clang::SourceLocation'].is_abstract = False
        asg['class ::clang::SourceLocation'].is_copyable = True
        asg['class ::clang::TemplateArgument'].is_abstract = False
        asg['class ::clang::TemplateArgument'].is_copyable = True
        for cls in ['::clang::FriendDecl', '::clang::CapturedDecl', '::clang::OMPThreadPrivateDecl',
                    '::clang::NonTypeTemplateParmDecl', '::clang::TemplateArgumentList', '::clang::ImportDecl',
                    '::clang::TemplateTemplateParmDecl', '::clang::CapturedDecl', '::clang::OMPThreadPrivateDecl',
                    '::clang::NonTypeTemplateParmDecl', '::clang::TemplateArgumentList', '::clang::ImportDecl',
                    '::clang::TemplateTemplateParmDecl']:
            asg['class ' + cls].is_abstract = False
        
    asg['class ::pybind11::object'].pybind11_export = True
    asg['class ::pybind11::list'].pybind11_export = True 
    asg['class ::pybind11::str'].pybind11_export = True 
        
    subset = []
    classes = [asg['class ::clang::QualType'],
               asg['class ::clang::Type'],
               asg['class ::clang::Decl']]
    subset += classes
    for cls in classes:
        subset += cls.subclasses(recursive=True)

    subset.append(asg['class ::llvm::StringRef'])

    subset.append(asg['class ::clang::Sema'])
    subset.append(asg['class ::clang::ASTUnit'])
    subset.append(asg['class ::clang::ASTContext'])
    subset.append(asg['class ::clang::SourceManager'])
    subset.append(asg['class ::clang::FileID'])

    subset.append(asg['class ::clang::SourceLocation'])

    subset.append(asg['class ::clang::CXXBaseSpecifier'])
    subset.append(asg['class ::clang::DeclContext'])
    subset.append(asg['class ::clang::TemplateArgument'])

    subset.append(asg['class ::clang::TemplateArgumentList'])
    for cls in subset:
        for ctr in cls.constructors():
            ctr.pybind11_export = False

    subset.append(asg['enum ::clang::Type::TypeClass'])
    subset.append(asg['enum ::clang::AccessSpecifier'])
    subset.append(asg['enum ::clang::LinkageSpecDecl::LanguageIDs'])
    subset.append(asg['enum ::clang::BuiltinType::Kind'])
    subset.append(asg['enum ::clang::TemplateArgument::ArgKind'])
    subset.append(asg['enum ::clang::Decl::Kind'])
    subset.extend(asg.nodes('::clanglite::build_ast_from_code_with_args'))

    for node in subset:
        node.pybind11_export = True
        
    for fct in asg['::clanglite'].functions():
        if not fct.localname == 'build_ast_from_code_with_args':
            fct.parent = fct.parameters[0].qualified_type.desugared_type.unqualified_type
        fct.pybind11_export = True
        
    for mtd in asg['class ::clang::ASTContext'].methods(pattern='.*getSourceManager.*'):
        if mtd.return_type.globalname == 'class ::clang::SourceManager &':
                mtd.pybind11_export = True
                break
                
    if autowig.parser.plugin == 'libclang':
        for node in (asg.functions(pattern='.*(llvm|clang).*_(begin|end)')
                     + asg.functions(pattern='::clang::CXXRecordDecl::getCaptureFields')
                     + asg.functions(pattern='.*(llvm|clang).*getNameAsString')
                     + asg.nodes('::clang::NamedDecl::getQualifiedNameAsString')
                     + asg.functions(pattern='.*::clang::ObjCProtocolDecl')
                     + asg.nodes('::clang::ObjCProtocolDecl::collectInheritedProtocolProperties')
                     + asg.nodes('::clang::ASTUnit::LoadFromASTFile')
                     + asg.nodes('::clang::ASTUnit::getCachedCompletionTypes')
                     + asg.nodes('::clang::ASTUnit::getBufferForFile')
                     + asg.nodes('::clang::CXXRecordDecl::getCaptureFields')
                     + asg.nodes('::clang::ASTContext::SectionInfos')
                     + asg.nodes('::clang::ASTContext::getAllocator')
                     + asg.nodes('::clang::ASTContext::getObjCEncoding.*')
                     + asg.nodes('::clang::ASTContext::getAllocator')
                     + asg.nodes('::clang::QualType::getAsString')
                     + asg.nodes('::clang::SourceLocation::printToString')
                     + asg['class ::llvm::StringRef'].methods()):
            node.pybind11_export = False
            
    if autowig.parser.plugin == 'clanglite':
        for mtd in asg['class ::clang::Decl'].methods():
            if mtd.localname == 'hasAttr':
                mtd.pybind11_export = False

    for decl in ['class ::clang::NamedDecl',
                 'class ::clang::ClassTemplateSpecializationDecl',
                 'class ::clang::TemplateArgument']:
        for mtd in asg[decl].methods():
            if mtd.localname == 'getName':
                mtd.pybind11_export = False
                
    import sys
    try:
        from path import path as Path
    except:
        from path import Path
       
    import platform
    if any(platform.win32_ver()):
        for header in (Path(sys.prefix)/'Library'/'include'/'clang').walkfiles('*.h'):
            asg[header.abspath()].is_external_dependency = False
    else:
        for header in (Path(sys.prefix)/'include'/'clang').walkfiles('*.h'):
            asg[header.abspath()].is_external_dependency = False
    return asg

This `controller` is then dynamically registered and used on the ASG.

In [10]:
%%time

autowig.controller['clanglite'] = clanglite_controller
autowig.controller.plugin = 'clanglite'
asg = autowig.controller(asg)

CPU times: user 2min 25s, sys: 99.9 ms, total: 2min 26s
Wall time: 2min 26s


In order to wrap a subset of the **Clang** library, we need to select the `boost_python_pattern` `generator` implementation.

In [11]:
%%time

autowig.generator.plugin = 'pybind11_pattern'
wrappers = autowig.generator(asg,
                             module = os.path.join(GIT_ROOT, 'src', 'py', '_clanglite.cpp'),
                             decorator = os.path.join(GIT_ROOT, 'src', 'py', 'clanglite', '_clanglite.py'),
                             closure = False)

KeyError: '_valid_pybind11_export'

In [ ]:
%debug

> /home/pfernique/Desktop/miniconda/envs/fp17/lib/python3.7/subprocess.py(347)check_call()
    345         if cmd is None:
    346             cmd = popenargs[0]
--> 347         raise CalledProcessError(retcode, cmd)
    348     return 0
    349 

ipdb> u
> <ipython-input-15-60e6461c46f3>(5)<module>()
      1 CONDA_RECIPE = os.path.join(GIT_ROOT, 'etc', 'conda', 'python-clanglite')
      2 import multiprocessing
      3 os.environ['CPU_COUNT'] = str(max(multiprocessing.cpu_count() - 2, 1))
      4 subprocess.check_call('conda build ' + CONDA_RECIPE + ' -c statiskit -c defaults --override-channels',
----> 5                       shell=True)



The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [12]:
%%time
 
wrappers.write()

NameError: name 'wrappers' is not defined

Here is the list of the generated wrappers (untracked files).

In [13]:
!git -C {GIT_ROOT} status

HEAD detached at origin/release/fp17
nothing to commit, working tree clean


And here, we present the wrappers generated for the `clang::Decl` class.

In [14]:
WRAPPER = os.path.join(GIT_ROOT, 'src', 'py',
                       'wrapper_a6aedb4654a55a40aeecf4b1dc5fcc98.cpp')
!pygmentize {WRAPPER}

Error: cannot read infile: [Errno 2] No such file or directory: '/home/pfernique/Desktop/develop/M2P2/FP17/share/git/ClangLite/src/py/wrapper_a6aedb4654a55a40aeecf4b1dc5fcc98.cpp'


Once the wrappers are written on the disk, the bingings must be compiled and installed.
This can be done using available **Conda** recipes.

In [15]:
CONDA_RECIPE = os.path.join(GIT_ROOT, 'etc', 'conda', 'python-clanglite')
import multiprocessing
os.environ['CPU_COUNT'] = str(max(multiprocessing.cpu_count() - 2, 1))
subprocess.check_call('conda build ' + CONDA_RECIPE + ' -c statiskit -c defaults --override-channels',
                      shell=True)

CalledProcessError: Command 'conda build /home/pfernique/Desktop/develop/M2P2/FP17/share/git/ClangLite/etc/conda/python-clanglite -c statiskit -c defaults --override-channels' returned non-zero exit status 1.

Here is a report concerning objects wrapped using this notebook.

In [ ]:
import fp17
fp17.report(asg)